In [1]:
from Magics import macro as magics
import xarray
import json


In [2]:

def coordinates(projection, z, x, y):
        dataset = xarray.open_dataset("tiles/bbox-%s.nc" % projection)        
        zoom = "bounding-box-%d" % z
        
        dims = dataset["bounding-box-%d" % z ].dims
        box = dataset["bounding-box-%d" % z ].isel({"x%d" % z : x, "y%d" %z : y} ).values
    
        return box


In [3]:
coordinates("4326", 1, 1, 0)

array([  0., -90., 180.,  90.])

In [4]:

def magics_plot(out, projection, z, x, y) :
    
    projections = {
        "4326" : "cylindrical",
        
        "32661" : "EPSG:32661"
    }
    print("EPSG:%s" % projection)
    width = 512
    cm = width/40
    magics.version()
    box = coordinates(projection, z,x,y)
    print (box)
    magics.silent()
    tile=magics.mmap( 
                super_page_x_length = cm*2,
				super_page_y_length = cm,
				page_x_length = cm,
				page_y_length = cm,
				subpage_x_length = cm,
				subpage_y_length = cm,
				subpage_x_position = 0.,
				subpage_y_position = 0.,
				page_frame=False,
				page_id_line = False,
				subpage_frame = True,
                subpage_map_projection = projections[projection] , 
                subpage_coordinates_system = "projection",
                subpage_lower_left_longitude = box[0],
                subpage_lower_left_latitude = box[1],
                subpage_upper_right_longitude = box[2],
                subpage_upper_right_latitude = box[3]
			    )
    notile=magics.mmap( 
				page_x_length = cm,
				page_y_length = cm,
                page_x_position = cm,
				subpage_x_length = cm,
				subpage_y_length = cm,
				subpage_x_position = 0.,
				subpage_y_position = 0.,
				page_frame=False,
				page_id_line = False,
                subpage_map_projection = projections[projection], 
                subpage_coordinates_system = "projection",
                subpage_lower_left_longitude = box[0],
                subpage_lower_left_latitude = box[1],
                subpage_upper_right_longitude = box[2],
                subpage_upper_right_latitude = box[3]
			    )
    data_tile = magics.mtile(grib_input_file_name = "tempe.grib", 
                        grib_tile_projection = "EPSG:%s" % projection,
                        grib_tile_z = z, 
                        grib_tile_x = x, 
                        grib_tile_y = y)
    
    data_wind_tile = magics.mtile(grib_input_file_name = "uv.grib", 
                        grib_tile_projection = "EPSG:%s" % projection,
                        grib_tile_z = z, 
                        grib_tile_x = x, 
                        grib_tile_y = y)
    data = magics.mgrib(grib_input_file_name = "tempe.grib", 
                        )
    
    data_wind = magics.mgrib(grib_input_file_name = "uv.grib", 
                        )
    wind = magics.mwind(  wind_thinning_factor = 25,
                          wind_flag_origin_marker =  "dot",
                          wind_flag_origin_marker_size =  0.1,
                          wind_field_type =  "flags",
                          wind_flag_colour = "black",
                          wind_flag_length =  0.3)
    contour = magics.mcont( {"contour_level_selection_type": "interval",
                          "contour_shade_colour_method": "list",
                          "contour_legend_text": "Contour shade (Range: -48 / 56)",
                          "contour_shade": "on",
                          "contour_shade_max_level": 56,
                          "contour_hilo": "off",
                          "contour": "off",
                          "contour_line_thickness": 3,
                          "contour_label": "off",
                          "grib_missing_value_indicator": 9999,
                          "contour_shade_method": "area_fill",
                          "contour_shade_min_level": -48,
                          "contour_interval": 4,
                          "contour_shade_colour_list": ["rgb(0,0,0.5)", "rgb(0,0,0.5)", 
                                                        "rgb(0,0,0.5)", "rgb(0,0,0.5)", 
                                                        "rgb(0,0,0.5)", "rgb(0,0,0.85)", 
                                                        "rgb(0.25,0,1)", "blue_purple", 
                                                        "greenish_blue", "blue_green", "bluish_green",
                                                        "yellow_green", "greenish_yellow", "yellow", 
                                                        "orangish_yellow", "orange_yellow", 
                                                        "yellowish_orange", "orange", "reddish_orange", 
                                                        "red_orange", "orangish_red", "red", 
                                                        "magenta", "magenta", "magenta", "magenta", 
                                                        "magenta" ]} )
    
    coast = magics.mcoast(map_coastline_colour = "black",
                map_grid =  "on", 
                map_grid_colour = "grey", 
                map_label  ="off")
    out.clear_output()
    with out:
        display(magics.plot(tile, data_tile, contour, data_wind_tile, wind, coast, magics.page(),
                       notile, data, contour, data_wind, wind, coast))
        
    


In [5]:

import ipywidgets as widgets

projections = {"32661" : {}, "4326" : {} }
zooms = []



for p in projections:
    
    dataset = xarray.open_dataset("tiles/bbox-%s.nc" % p)
    zooms = dataset["zooms"].values
    
    for z in zooms:        
        x = dataset["x%d" % z ].values
        y = dataset[ "y%d" % z ].values        
        projections[p]["zoom-level-%d" % z] = { "x" : x, "y" : y}




#dir(widgets)


out = widgets.Output(layout={'border': '1px solid black', 'height':'800px', 'width':'1200px'})
projection = widgets.Dropdown(description='Projection', labels = ["xx", "YY"], options= projections.keys())
zoom = widgets.IntSlider(description='Zoom level', min =1, max =6)
x = widgets.IntSlider(description='X')
y = widgets.IntSlider(description='Y')


def change_projection(change):
    x.min = projections[change.new]['zoom-level-%d' % zoom.value]["x"][0]
    x.max = projections[change.new]['zoom-level-%d' % zoom.value]["x"][-1]
    y.min = projections[change.new]['zoom-level-%d' % zoom.value]["y"][0]
    y.max = projections[change.new]['zoom-level-%d' % zoom.value]["y"][-1]

def change_zoom(change):
    x.min = projections[projection.value]['zoom-level-%d' % change.new]["x"][0]
    x.max = projections[projection.value]['zoom-level-%d' % change.new]["x"][-1]
    y.min = projections[projection.value]['zoom-level-%d' % change.new]["y"][0]
    y.max = projections[projection.value]['zoom-level-%d' % change.new]["y"][-1]
    
projection.observe(change_projection, names='value')
zoom.observe(change_zoom, names='value')


def plot_projection(change):
    magics_plot(out, change.new, zoom.value, x.value, y.value)

def plot_zoom(change):
    magics_plot(out, projection.value, change.new, x.value, y.value)

def plot_x(change):
    magics_plot(out, projection.value, zoom.value, change.new, y.value)

def plot_y(change):
    magics_plot(out, projection.value, zoom.value, x.value, change.new)
    
projection.observe(plot_projection, names='value')
zoom.observe(plot_zoom, names='value')
x.observe(plot_x, names='value')
y.observe(plot_y, names='value')


display(projection, zoom, x, y)
display(out)



Dropdown(description='Projection', options=('32661', '4326'), value='32661')

IntSlider(value=1, description='Zoom level', max=6, min=1)

IntSlider(value=0, description='X')

IntSlider(value=0, description='Y')

Output(layout=Layout(border='1px solid black', height='800px', width='1200px'))